# Necessary Imports

In [118]:
import gzip
import json
import pandas as pd
import re
import numpy as np
import random

# Reading the Data and Splitting Them into Two Sets

Create two sets: One is tweets tweeted before 30 November, 2019, the other is after 16 March 2020.

In [119]:
tweets_expanded = pd.read_csv("/content/drive/MyDrive/tweets_df.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,13,15,17,18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
tweets_expanded.head(5)

,Unnamed: 0,location,description,name,screen_name,id_str,created_at,province_codes,genders,following,...,followers_count,following_count,downloaded,ref_twt_id_str,ref_twt_txt,ref_usr_id_str,type,twt_date,twt_id_str,twt_txt
0,0,"istanbul, türkiye",karınca yükünü fil çekmez oldu..,yılmaz şengül,deatime,31109031,Tue Apr 14 12:22:59 +0000 2009,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","['1481278402514636806', '15144122', '104272079...",...,1851.0,1099.0,220318,1.481613e+18,"Ekmek küçülüyor, hem kamuda hem özelde… \n\nTü...",1.481278e+18,retweet,220113,NaN,NaN
1,1,"istanbul, türkiye",karınca yükünü fil çekmez oldu..,yılmaz şengül,deatime,31109031,Tue Apr 14 12:22:59 +0000 2009,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","['1481278402514636806', '15144122', '104272079...",...,1851.0,1099.0,220318,1.481568e+18,Enflasyon Oranında Iyileştirme istiyoruz .\n\n...,1.081091e+18,retweet,220113,NaN,NaN
2,2,"istanbul, türkiye",karınca yükünü fil çekmez oldu..,yılmaz şengül,deatime,31109031,Tue Apr 14 12:22:59 +0000 2009,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","['1481278402514636806', '15144122', '104272079...",...,1851.0,1099.0,220318,1.481489e+18,"Biz Taşeronuz , O Eşek KADROLU. \n\n450 Bin Be...",1.081091e+18,retweet,220113,NaN,NaN
3,3,"istanbul, türkiye",karınca yükünü fil çekmez oldu..,yılmaz şengül,deatime,31109031,Tue Apr 14 12:22:59 +0000 2009,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","['1481278402514636806', '15144122', '104272079...",...,1851.0,1099.0,220318,NaN,NaN,NaN,original,220112,1.481377e+18,Efsaneler Ölmez ..\n\n#Lefter https://t.co/bl9...
4,4,"istanbul, türkiye",karınca yükünü fil çekmez oldu..,yılmaz şengül,deatime,31109031,Tue Apr 14 12:22:59 +0000 2009,"[{'source': 'location', 'pcode': 34}]","[{'source': 'name', 'gender': 'male'}]","['1481278402514636806', '15144122', '104272079...",...,1851.0,1099.0,220318,NaN,NaN,NaN,original,220112,1.481341e+18,HALK 'ın Ekmeğidir ADALET ..\n\nSiz Vermeyecek...


In [ ]:
tweets_expanded.columns

Index(['Unnamed: 0', 'location', 'description', 'name', 'screen_name',
       'id_str', 'created_at', 'province_codes', 'genders', 'following',
       'followers', 'followers_count', 'following_count', 'downloaded',
       'ref_twt_id_str', 'ref_twt_txt', 'ref_usr_id_str', 'type', 'twt_date',
       'twt_id_str', 'twt_txt'],
      dtype='object')

In [120]:
tweets_expanded = tweets_expanded.drop(tweets_expanded.columns[[0,1,2,3,4, 5,6,7,8,9, 10,11,12,13, 14,16,19]], axis=1) 

Size of the dataset:

In [121]:
len_pre = 10000
len_pandemic = 10000

In [122]:
pre_covid = pd.DataFrame(columns=list(tweets_expanded.columns))
pandemic = pd.DataFrame(columns=list(tweets_expanded.columns))

for i, date in enumerate(tweets_expanded["twt_date"]):
    if int(date) < 191200 and len(pre_covid) < len_pre:
        pre_covid = pre_covid.append(tweets_expanded.loc[i,:])
    elif int(date) > 200315 and len(pandemic) < len_pandemic:
        pandemic = pandemic.append(tweets_expanded.loc[i,:])
    if (len(pre_covid) + len(pandemic)) == (len_pre + len_pandemic):
        break

In [28]:
pre_covid.tail()

,ref_twt_txt,type,twt_date,twt_txt
29508,"Be any #colour you choose to be, be yourself, ...",fav,190606,NaN
29621,"allaha sukur gonlumuz genıs, ask insanıyız, at...",fav,190305,NaN
29623,Empati yapmak başkasının yerine de düşünebilme...,fav,160103,NaN
29691,NaN,original,170208,https://t.co/7cQMeuAir9
29692,NaN,reply,161208,@turk_videolar


In [29]:
tweets_expanded.loc[247,:]

ref_twt_txt                        NaN
type                          original
twt_date                        190611
twt_txt        https://t.co/ssEEDk8oPe
Name: 247, dtype: object

In [123]:
pre_covid_tweets = []
pandemic_tweets = []

for i, Bool in pre_covid.isnull()["twt_txt"].items():
    if Bool:
        pre_covid_tweets.append(pre_covid.loc[i,"ref_twt_txt"])
    else:
        pre_covid_tweets.append(pre_covid.loc[i,"twt_txt"])

for i, Bool in pandemic.isnull()["twt_txt"].items():
    if Bool:
        pandemic_tweets.append(pandemic.loc[i,"ref_twt_txt"])
    else:
        pandemic_tweets.append(pandemic.loc[i,"twt_txt"])        

In [124]:
random.shuffle(pre_covid_tweets)
random.shuffle(pandemic_tweets)

In [125]:
pre_covid_tweets = pre_covid_tweets[:500]
pandemic_tweets = pandemic_tweets[:500]

In [ ]:
pre_covid_tweets

In [ ]:
pandemic_tweets

# Sentence Splitting

In [126]:
pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [127]:
from emoji import UNICODE_EMOJI

In [128]:
separators = ["\n", ".", ":", "!", "?"]

In [129]:
def no_link(tweet):    
    ind1 = tweet.find(' ')
    ind2 = tweet.find('\n')
    if ind1 == -1:
        ind1 = 10000
    if ind2 == -1:
        ind2 = 10000
    if ind1 < ind2:
        tweet = tweet[ind1:]
    elif ind1 > ind2:
        tweet = tweet[ind2:]
    else:
        tweet = ''
    return tweet

In [130]:
pre_covid_sentences = []
link_headers = ['http://t.co/','https://t.co/']
for i, tweet in enumerate(pre_covid_tweets):
    tweets_wo_link = []
    x = 0
    y = 0    
    if link_headers[0] in tweet:
        x = 1
    if link_headers[1] in tweet:
        y = 1
    while tweet:
        if x == 0 and y == 0:
            tweets_wo_link.append(tweet)
            tweet = False
        elif x == 0 and y == 1:
            ind = tweet.find(link_headers[1]) 
            tweets_wo_link.append(tweet[0:ind])
            tweet = tweet[ind:]
            tweet = no_link(tweet)
                
            if link_headers[1] not in tweet:
                y = 0
                
        elif x == 1 and y == 0:
            ind = tweet.find(link_headers[0])
            tweets_wo_link.append(tweet[0:ind])
            tweet = tweet[ind:]
            tweet = no_link(tweet)
            if link_headers[0] not in tweet:
                x = 0
               
        elif x == 1 and y == 1:
            ind0 = tweet.find(link_headers[0])
            ind1 = tweet.find(link_headers[1])
            if ind0 < ind1:
                ind = tweet.find(link_headers[0])
                tweets_wo_link.append(tweet[0:ind])
                tweet = tweet[ind:]
                tweet = no_link(tweet)
            elif ind0 > ind1:
                ind = tweet.find(link_headers[1])
                tweets_wo_link.append(tweet[0:ind])
                tweet = tweet[ind:]
                tweet = no_link(tweet)            
            if link_headers[1] not in tweet:
                y = 0
            if link_headers[0] not in tweet:
                x = 0
    tweets_wo_link = [x for x in tweets_wo_link if len(x) != 0]
    sentences = []
    j = 0
    for tweet_wo_link in tweets_wo_link:
        for i, char in enumerate(tweet_wo_link):
            if char in UNICODE_EMOJI['en'] or char in separators:
                sentences.append(tweet_wo_link[j:i])
                j = i + 1
        sentences.append(tweet_wo_link[j:])
    sentences = [x for x in sentences if len(x) != 0]
    pre_covid_sentences.append(sentences)

In [131]:
pandemic_sentences = []
link_headers = ['http://t.co/','https://t.co/']
for tweet in pandemic_tweets:
    tweets_wo_link = []
    x = 0
    y = 0    
    if link_headers[0] in tweet:
        x = 1
    if link_headers[1] in tweet:
        y = 1
    while tweet:
        if x == 0 and y == 0:
            tweets_wo_link.append(tweet)
            tweet = False
        elif x == 0 and y == 1:
            ind = tweet.find(link_headers[1]) 
            tweets_wo_link.append(tweet[0:ind])
            tweet = tweet[ind:]
            tweet = no_link(tweet)
                
            if link_headers[1] not in tweet:
                y = 0
                
        elif x == 1 and y == 0:
            ind = tweet.find(link_headers[0])
            tweets_wo_link.append(tweet[0:ind])
            tweet = tweet[ind:]
            tweet = no_link(tweet)
            if link_headers[0] not in tweet:
                x = 0
               
        elif x == 1 and y == 1:
            ind0 = tweet.find(link_headers[0])
            ind1 = tweet.find(link_headers[1])
            if ind0 < ind1:
                ind = tweet.find(link_headers[0])
                tweets_wo_link.append(tweet[0:ind])
                tweet = tweet[ind:]
                tweet = no_link(tweet)
            elif ind0 > ind1:
                ind = tweet.find(link_headers[1])
                tweets_wo_link.append(tweet[0:ind])
                tweet = no_link(tweet)
            ind0 = tweet.find(link_headers[0])
            ind1 = tweet.find(link_headers[1])
            if link_headers[1] not in tweet:
                y = 0
            if link_headers[0] not in tweet:
                x = 0
    tweets_wo_link = [x for x in tweets_wo_link if len(x) != 0]
    sentences = []
    j = 0
    for tweet_wo_link in tweets_wo_link:
        for i, char in enumerate(tweet_wo_link):
            if char in UNICODE_EMOJI['en'] or char in separators:
                sentences.append(tweet_wo_link[j:i])
                j = i + 1
        sentences.append(tweet_wo_link[j:])
    sentences = [x for x in sentences if len(x) != 0]
    pandemic_sentences.append(sentences)

In [42]:
len(pre_covid_sentences)

500

In [43]:
len(pandemic_sentences)

500

In [ ]:
pre_covid_sentences

# Morphological Segmentation

From Emre Can Açıkgöz's private code, we tag the tweets and get the lists 'pre_covid_tagged' and 'pandemic_tagged.'

In [ ]:
pre_covid_tagged

In [ ]:
pandemic_tagged

# Feature Extraction

In [132]:
no_features = 20

In [133]:
pre_covid_features = np.zeros((len(pre_covid_tweets), no_features))
pandemic_features = np.zeros((len(pandemic_tweets), no_features))

Extract Feature 1: All Punctuation

In [46]:
punctuations = ['.', ',', ':', ';']

for i, tweet in enumerate(pre_covid_tweets):
    x = 0
    for punctuation in punctuations:
        x += tweet.count(punctuation)
    pre_covid_features[i, 0] = x

for i, tweet in enumerate(pandemic_tweets):
    x = 0
    for punctuation in punctuations:
        x += tweet.count(punctuation)
    pandemic_features[i, 0] = x

Extract Feature 2: Number of Commas

In [47]:
for i, tweet in enumerate(pre_covid_tweets):
    pre_covid_features[i, 1] = tweet.count(',')

for i, tweet in enumerate(pandemic_tweets):
    pandemic_features[i, 1] = tweet.count(',')

Extract Feature 3: Number of '@' patterns

In [48]:
for i, tweet in enumerate(pre_covid_tweets):
    pre_covid_features[i, 2] = len(re.findall(r"@\S+", tweet))

for i, tweet in enumerate(pandemic_tweets):
    pandemic_features[i, 2] = len(re.findall(r"@\S+", tweet))

Extract Feature 4: Number of '!'

In [49]:
for i, tweet in enumerate(pre_covid_tweets):
    pre_covid_features[i, 3] = tweet.count('!')

for i, tweet in enumerate(pandemic_tweets):
    pandemic_features[i, 3] = tweet.count('!')

Extract Feature 5: Number of External Links

In [50]:
for i, tweet in enumerate(pre_covid_tweets):
    pre_covid_features[i, 4] = tweet.count('http://t.co/') + tweet.count('https://t.co/')

for i, tweet in enumerate(pandemic_tweets):
    pandemic_features[i, 4] = tweet.count('http://t.co/') + tweet.count('https://t.co/')

Extract Feature 6: Number of first person singular pronouns

In [51]:
for i, tweet in enumerate(pre_covid_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'A1sg' in word:
                pre_covid_features[i, 5] += 1
for i, tweet in enumerate(pandemic_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'A1sg' in word:
                pandemic_features[i, 5] += 1

Extract Feature 7 Using the Morphological Analyzer: the count of negative particles (değil, -me and -ma)

In [52]:
for i, tweet in enumerate(pre_covid_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'Neg' in word:
                pre_covid_features[i, 6] += 1
for i, tweet in enumerate(pandemic_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'Neg' in word:
                pandemic_features[i, 6] += 1

Extract Feture 8: Number of numbers

In [53]:
for i, tweet in enumerate(pre_covid_tweets):
    pre_covid_features[i, 7] = sum(c.isdigit() for c in tweet)

for i, tweet in enumerate(pandemic_tweets):
    pandemic_features[i, 7] = sum(c.isdigit() for c in tweet)

Extract Feature 9: Number of parenthetical phrases

In [54]:
for i, tweet in enumerate(pre_covid_tweets):
    pre_covid_features[i, 8] = len(re.findall(r'\(.+?\)', tweet))

for i, tweet in enumerate(pandemic_tweets):
    pandemic_features[i, 8] = len(re.findall(r'\(.+?\)', tweet))

Extract Feature 10: Number of prepositions

In [55]:
for i, tweet in enumerate(pre_covid_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'Postp' in word:
                pre_covid_features[i, 9] += 1
for i, tweet in enumerate(pandemic_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'Postp' in word:
                pandemic_features[i, 9] += 1

Extract Feature 11: Number of pronouns

In [56]:
for i, tweet in enumerate(pre_covid_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'Pers' in word:
                pre_covid_features[i, 10] += 1
            if 'Pron' in word:
                pre_covid_features[i, 10] += 1
            if 'Pnon' in word:
                pre_covid_features[i, 10] += 1
            if 'Reflex' in word:
                pre_covid_features[i, 10] += 1
for i, tweet in enumerate(pandemic_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'Pers' in word:
                pandemic_features[i, 10] += 1
            if 'Pron' in word:
                pandemic_features[i, 10] += 1
            if 'Pnon' in word:
                pandemic_features[i, 10] += 1
            if 'Reflex' in word:
                pandemic_features[i, 10] += 1

Extract Feature 12: Number of '?'

In [57]:
for i, tweet in enumerate(pre_covid_tweets):
    pre_covid_features[i, 11] = tweet.count('?')
    
for i, tweet in enumerate(pandemic_tweets):
    pandemic_features[i, 11] = tweet.count('?')

Extract Feature 13: Number of words longer than 6 letters

In [58]:
for i, tweet in enumerate(pre_covid_tweets):
    x = 0
    lengths = [len(word) for word in tweet.split()]
    for length in lengths:
        if length > 6:
            x += 1
    pre_covid_features[i, 12] = x
    
for i, tweet in enumerate(pandemic_tweets):
    x = 0
    lengths = [len(word) for word in tweet.split()]
    for length in lengths:
        if length > 6:
            x += 1
    pandemic_features[i, 12] = x

Extract Feature 14: Number of first person (singular and plural) pronouns

In [59]:
for i, tweet in enumerate(pre_covid_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'A1sg' in word:
                pre_covid_features[i, 13] += 1
            if 'P1sg' in word:
                pre_covid_features[i, 13] += 1
            if 'A1pl' in word:
                pre_covid_features[i, 13] += 1
            if 'P1pl' in word:
                pre_covid_features[i, 13] += 1
for i, tweet in enumerate(pandemic_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'A1sg' in word:
                pandemic_features[i, 13] += 1
            if 'P1sg' in word:
                pandemic_features[i, 13] += 1
            if 'A1pl' in word:
                pandemic_features[i, 13] += 1
            if 'P1pl' in word:
                pandemic_features[i, 13] += 1

Extract Feature 15: Type/token ratio (tt)

In [60]:
for i, tweet in enumerate(pre_covid_tweets):
    words = tweet.lower().split()
    tt = len(set(words)) / len(words)
    pre_covid_features[i, 14] = tt
    
for i, tweet in enumerate(pandemic_tweets):
    words = tweet.lower().split()
    tt = len(set(words)) / len(words)
    pandemic_features[i, 14] = tt

Extract Feature 16: Number of words

In [61]:
for i, tweet in enumerate(pre_covid_tweets):
    pre_covid_features[i, 15] = len(tweet.split())

for i, tweet in enumerate(pandemic_tweets):
    pandemic_features[i, 15] = len(tweet.split())

Extract Feature 17: Number of first person plural pronouns

In [62]:
for i, tweet in enumerate(pre_covid_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'A1pl' in word:
                pre_covid_features[i, 16] += 1
            if 'P1pl' in word:
                pre_covid_features[i, 16] += 1
for i, tweet in enumerate(pandemic_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'A1pl' in word:
                pandemic_features[i, 16] += 1
            if 'P1pl' in word:
                pandemic_features[i, 16] += 1

In [ ]:
"""fpps = ["Biz", "Biz", "bız", "biz"]
for i, tweet in enumerate(pre_covid_tweets):
    x = 0
    for fpp in fpps:
        x += tweet.count(fpp)
    pre_covid_features[i, 16] = x

for i, tweet in enumerate(pandemic_tweets):
    x = 0
    for fpp in fpps:
        x += tweet.count(fpp)
    pandemic_features[i, 16] = x"""

Extract Feature 18: Number of second person singular pronouns

In [63]:
for i, tweet in enumerate(pre_covid_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'A2sg' in word:
                pre_covid_features[i, 17] += 1
            if 'P2sg' in word:
                pre_covid_features[i, 17] += 1
for i, tweet in enumerate(pandemic_tagged):
    for sentence in tweet:
        for word in sentence:
            if 'A2sg' in word:
                pandemic_features[i, 17] += 1
            if 'P2sg' in word:
                pandemic_features[i, 17] += 1

In [ ]:
"""spss = ["Sen", "sen"]
for i, tweet in enumerate(pre_covid_tweets):
    x = 0
    for sps in spss:
        x += tweet.count(sps)
    pre_covid_features[i, 17] = x

for i, tweet in enumerate(pandemic_tweets):
    x = 0
    for sps in spss:
        x += tweet.count(sps)
    pandemic_features[i, 17] = x"""

Extract Feature 19: Simple mean of the frequency (mf) of words

In [64]:
pre_covid_vocabulary = {}
for tweet in pre_covid_tweets:
    words = tweet.lower().split()
    for word in words:
        pre_covid_vocabulary[word] = pre_covid_vocabulary.get(word, 0) + 1
        
pandemic_vocabulary = {}
for tweet in pandemic_tweets:
    words = tweet.lower().split()
    for word in words:
        pandemic_vocabulary[word] = pandemic_vocabulary.get(word, 0) + 1
        
for i, tweet in enumerate(pre_covid_tweets):
    words = tweet.lower().split()
    sum_of_frequencies = 0.0
    for word in words:
        sum_of_frequencies += pre_covid_vocabulary.get(word, 1)
    mf = sum_of_frequencies / len(words)
    pre_covid_features[i, 18] = mf
for i, tweet in enumerate(pandemic_tweets):
    words = tweet.lower().split()
    sum_of_frequencies = 0.0
    for word in words:
        sum_of_frequencies += pandemic_vocabulary.get(word, 1)
    mf = sum_of_frequencies / len(words)
    pandemic_features[i, 18] = mf   

Feature 20: Number of words mentioned from the "Extraverted Words" list

In [65]:
%cd ..

/content/drive/MyDrive


In [135]:
extraverted_words = []
with open('extraverted_words.txt', 'r') as f:
    for line in f.readlines():
        extraverted_words.append(line.rstrip())
extraverted_words

['beraber',
 'birlikte',
 'ekip',
 'takım',
 'dost',
 'sosyal',
 'arkadaş',
 'buluş',
 'paylaş',
 'bir arada',
 'bir araya']

In [136]:
for i, tweet in enumerate(pre_covid_tweets):
    for word in extraverted_words:
        if word in tweet:
            pre_covid_features[(i,19)] += 1

for i, tweet in enumerate(pandemic_tweets):
    for word in extraverted_words:
        if word in tweet:
            pandemic_features[(i,19)] += 1

# Calculate the Means and Standard Deviations of the Features

In [137]:
mean_pre_covid = np.mean(pre_covid_features, axis=0)
mean_pandemic = np.mean(pandemic_features, axis=0)

In [138]:
std_pre_covid = np.std(pre_covid_features, axis=0)
std_pandemic = np.std(pandemic_features, axis=0)

In [139]:
threshold_pre_covid = mean_pre_covid + std_pre_covid
threshold_pandemic = mean_pandemic + std_pandemic

# Compute the Extraversion Scores

In [140]:
neg_correlation_indices = []
pos_correlation_indices = [19]

In [141]:
pos_correlation_pre = pre_covid_features[:, pos_correlation_indices]
neg_correlation_pre = pre_covid_features[:, neg_correlation_indices]
pos_correlation_pandemic = pandemic_features[:, pos_correlation_indices]
neg_correlation_pandemic = pandemic_features[:, neg_correlation_indices]

In [142]:
pos_scores_pre = np.sum(pos_correlation_pre >= threshold_pre_covid[pos_correlation_indices], axis=1)
neg_scores_pre = np.sum(neg_correlation_pre >= threshold_pre_covid[neg_correlation_indices],axis=1)
pos_scores_pandemic = np.sum(pos_correlation_pandemic >= threshold_pandemic[pos_correlation_indices], axis=1)
neg_scores_pandemic = np.sum(neg_correlation_pandemic >= threshold_pandemic[neg_correlation_indices],axis=1)

In [143]:
scores_pre = pos_scores_pre - neg_scores_pre
scores_pandemic = pos_scores_pandemic - neg_scores_pandemic

In [99]:
neg_correlation_indices_high = [0, 2, 6, 8, 11, 12, 14]
neg_correlation_indices_moderate = [4]
neg_correlation_indices_weak = [1, 7, 15, 17]
pos_correlation_indices_high = [10, 13, 16, 19]
pos_correlation_indices_moderate = [5, 18]

In [100]:
pos_scores_pre_h = np.sum(pre_covid_features[:, pos_correlation_indices_high] >= threshold_pre_covid[pos_correlation_indices_high], axis=1)
pos_scores_pre_m = np.sum(pre_covid_features[:, pos_correlation_indices_moderate] >= threshold_pre_covid[pos_correlation_indices_moderate], axis=1)
neg_scores_pre_h = np.sum(pre_covid_features[:, neg_correlation_indices_high] >= threshold_pre_covid[neg_correlation_indices_high], axis=1)
neg_scores_pre_m = np.sum(pre_covid_features[:, neg_correlation_indices_moderate] >= threshold_pre_covid[neg_correlation_indices_moderate], axis=1)
neg_scores_pre_w = np.sum(pre_covid_features[:, neg_correlation_indices_weak] >= threshold_pre_covid[neg_correlation_indices_weak], axis=1)
pos_scores_pandemic_h = np.sum(pandemic_features[:, pos_correlation_indices_high] >= threshold_pandemic[pos_correlation_indices_high], axis=1)
pos_scores_pandemic_m = np.sum(pandemic_features[:, pos_correlation_indices_moderate] >= threshold_pandemic[pos_correlation_indices_moderate], axis=1)
neg_scores_pandemic_h = np.sum(pandemic_features[:, neg_correlation_indices_high] >= threshold_pandemic[neg_correlation_indices_high], axis=1)
neg_scores_pandemic_m = np.sum(pandemic_features[:, neg_correlation_indices_moderate] >= threshold_pandemic[neg_correlation_indices_moderate], axis=1)
neg_scores_pandemic_w = np.sum(pandemic_features[:, neg_correlation_indices_weak] >= threshold_pandemic[neg_correlation_indices_weak], axis=1)

In [113]:
scores_pre = 5*pos_scores_pre_h + 4*pos_scores_pre_m - 5*neg_scores_pre_h -4*neg_scores_pre_m - 3*neg_scores_pre_w
scores_pandemic = 5*pos_scores_pandemic_h +4*pos_scores_pandemic_m - 5*neg_scores_pandemic_h - 4*neg_scores_pandemic_m -3*neg_scores_pandemic_w

In [144]:
scores_pre

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [145]:
scores_pandemic

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [146]:
pre_covid_extraversion = np.mean(scores_pre)
pandemic_extraversion = np.mean(scores_pandemic)
print("Average extraversion score for tweets shared before the pandemic: ", pre_covid_extraversion)
print("Average extraversion score for tweets shared during the pandemic: ", pandemic_extraversion)

Average extraversion score for tweets shared before the pandemic:  0.048
Average extraversion score for tweets shared during the pandemic:  0.06
